<a href="https://colab.research.google.com/github/dariashcherbakovaaa/Algorithms-for-massive-data/blob/main/Algorithms_for_massive_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Market-Basket Analysis on LinkedIn job description
### *DSE student, 17487A*
### *Daria Shcherbakova*


# Environment settings & data downloading

In [11]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import os
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName('PageRank').getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark
import pyspark
type(spark)

sc = spark.sparkContext
spark

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


❗️❗️❗️❗️❗️❗️❗️❗️❗️❗️❗️❗️❕❕❕ **ATTENTION** ❕❕❕❗️❗️❗️❗️❗️❗️❗️❗️❗️❗️❗️❗️

❗️❗️❗️❗️❗️❗️❗️❗️❗️❗️❗️❗️❕❕❕❗️❗️❗️❗️❗️❗️❗️❕❕❕❗️❗️❗️❗️❗️❗️❗️❗️❗️❗️❗️❗️



**I use *google.colab.userdata* for secret key to activate kaggle**



❗️❗️❗️❗️❗️❗️❗️❗️❗️❗️❗️❗️❕❕❕❗️❗️❗️❗️❗️❗️❗️❕❕❕❗️❗️❗️❗️❗️❗️❗️❗️❗️❗️❗️❗️

❗️❗️❗️❗️❗️❗️❗️❗️❗️❗️❗️❗️❕❕❕ **ATTENTION** ❕❕❕❗️❗️❗️❗️❗️❗️❗️❗️❗️❗️❗️❗️

In [12]:
from google.colab import userdata
os.environ['KAGGLE_USERNAME'] = userdata.get("KAGGLE_USERNAME")
os.environ['KAGGLE_KEY'] = userdata.get("KAGGLE_KEY")

In [13]:
# LinkedIn dataset (Finding similar items, Market-basket analysis)
!kaggle datasets download -d asaniczka/1-3m-linkedin-jobs-and-skills-2024
!unzip 1-3m-linkedin-jobs-and-skills-2024.zip -d job_skills

Dataset URL: https://www.kaggle.com/datasets/asaniczka/1-3m-linkedin-jobs-and-skills-2024
License(s): ODC Attribution License (ODC-By)
100% 1.88G/1.88G [01:16<00:00, 29.2MB/s]
100% 1.88G/1.88G [01:16<00:00, 26.3MB/s]
Archive:  1-3m-linkedin-jobs-and-skills-2024.zip
  inflating: job_skills/job_skills.csv  
  inflating: job_skills/job_summary.csv  
  inflating: job_skills/linkedin_job_postings.csv  


In [14]:
data_path = '/content/job_skills/job_skills.csv'
df_skills = spark.read.csv(data_path, inferSchema=True, header=True)
df_skills.printSchema()
df_skills.show()

root
 |-- job_link: string (nullable = true)
 |-- job_skills: string (nullable = true)

+--------------------+--------------------+
|            job_link|          job_skills|
+--------------------+--------------------+
|https://www.linke...|Building Custodia...|
|https://www.linke...|Customer service,...|
|https://www.linke...|Applied Behavior ...|
|https://www.linke...|Electrical Engine...|
|https://www.linke...|Electrical Assemb...|
|https://www.linke...|Access Control, V...|
|https://www.linke...|Consultation, Sup...|
|https://www.linke...|Veterinary Recept...|
|https://www.linke...|Optical Inspectio...|
|https://www.linke...|HVAC, troubleshoo...|
|https://www.linke...|Host/Server Assis...|
|https://www.linke...|Apartment mainten...|
|https://www.linke...|Fiber Optic Cable...|
|https://www.linke...|CT Technologist, ...|
|https://ca.linked...|SAP, DRMIS, Data ...|
|https://www.linke...|Debt and equity o...|
|https://ca.linked...|Biomedical Engine...|
|https://www.linke...|Laboratory

In [15]:
data_path = '/content/job_skills/linkedin_job_postings.csv'
df_all = spark.read.csv(data_path, inferSchema=True, header=True)
df_all.printSchema()
df_all.show()

root
 |-- job_link: string (nullable = true)
 |-- last_processed_time: string (nullable = true)
 |-- got_summary: string (nullable = true)
 |-- got_ner: string (nullable = true)
 |-- is_being_worked: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- company: string (nullable = true)
 |-- job_location: string (nullable = true)
 |-- first_seen: string (nullable = true)
 |-- search_city: string (nullable = true)
 |-- search_country: string (nullable = true)
 |-- search_position: string (nullable = true)
 |-- job_level: string (nullable = true)
 |-- job_type: string (nullable = true)

+--------------------+--------------------+-----------+-------+---------------+--------------------+--------------------+--------------------+----------+---------------+--------------+--------------------+----------+--------+
|            job_link| last_processed_time|got_summary|got_ner|is_being_worked|           job_title|             company|        job_location|first_seen|    search_c

In [ ]:
os.remove('/content/1-3m-linkedin-jobs-and-skills-2024.zip')
os.remove('/content/job_skills/job_skills.csv')
os.remove('/content/job_skills/job_summary.csv')
os.remove('/content/job_skills/linkedin_job_postings.csv')
!rmdir /content/job_skills/

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Data pre-processing

##### The main dataset with links and skills

In [16]:
new_names = ['link', 'skill']

df_skills = df_skills.toDF(*new_names)
df_skills.limit(5).toPandas()

,link,skill
0,https://www.linkedin.com/jobs/view/housekeeper...,"Building Custodial Services, Cleaning, Janitor..."
1,https://www.linkedin.com/jobs/view/assistant-g...,"Customer service, Restaurant management, Food ..."
2,https://www.linkedin.com/jobs/view/school-base...,"Applied Behavior Analysis (ABA), Data analysis..."
3,https://www.linkedin.com/jobs/view/electrical-...,"Electrical Engineering, Project Controls, Sche..."
4,https://www.linkedin.com/jobs/view/electrical-...,"Electrical Assembly, Point to point wiring, St..."


In [17]:
df_skills.describe() # check the number of cells for all columns

summary,link,skill
count,1296381,1294374
mean,null,null
stddev,null,null
min,https://ae.linked...,"""* Maintenance, *..."
max,https://za.linked...,"志愿服务, 沟通, 陪伴, 临终服..."


In [18]:
df_skills = df_skills.dropna() # drop empty, NA cells
df_skills = df_skills.dropDuplicates() # drop repeated cells

df_skills.describe()

summary,link,skill
count,1294374,1294374
mean,null,null
stddev,null,null
min,https://ae.linked...,"""* Maintenance, *..."
max,https://za.linked...,"志愿服务, 沟通, 陪伴, 临终服..."


In [ ]:
df_skills.groupBy('skill').count().orderBy('count', ascending=False).show() # first try to check the most "frequent" skills in dataset

##### The additional dataset for sampling

In [21]:
df_all = df_all[['job_link', 'job_title', 'search_country', 'search_position', 'job_level']]
df_all.show(10)

+--------------------+--------------------+--------------+--------------------+----------+
|            job_link|           job_title|search_country|     search_position| job_level|
+--------------------+--------------------+--------------+--------------------+----------+
|https://www.linke...|Account Executive...| United States|         Color Maker|Mid senior|
|https://www.linke...|Registered Nurse ...| United States|Director Nursing ...|Mid senior|
|https://www.linke...|RESTAURANT SUPERV...| United States|            Stand-In|Mid senior|
|https://www.linke...|Independent Real ...| United States|   Real-Estate Clerk|Mid senior|
|https://www.linke...|Group/Unit Superv...| United States|Supervisor Travel...|Mid senior|
|https://www.linke...|  Counselor-07532802| United States| Residence Counselor|Mid senior|
|https://www.linke...|Registered Nurse ...| United States|  Nurse Practitioner|Mid senior|
|https://www.linke...|Part Time- HR Gen...| United States|Human Resource Ad...|Mid senior|

In [22]:
df_all.describe()

summary,job_link,job_title,search_country,search_position,job_level
count,1348488,1348488,1348420,1348420,1348420
mean,null,null,null,null,null
stddev,null,null,null,null,null
min,"New""","""""""A"""" Softball C...",2024-01-13,Able Seaman,Administrative As...
max,https://za.linked...,"🔥Nurse Manager, ...",United States,Zoo Veterinarian,United States


In [23]:
df_all = df_all.where((df_all['job_level'] == 'Mid senior') & (df_all['search_country'] == 'United States'))
df_all.show(10)

+--------------------+--------------------+--------------+--------------------+----------+
|            job_link|           job_title|search_country|     search_position| job_level|
+--------------------+--------------------+--------------+--------------------+----------+
|https://www.linke...|Account Executive...| United States|         Color Maker|Mid senior|
|https://www.linke...|Registered Nurse ...| United States|Director Nursing ...|Mid senior|
|https://www.linke...|RESTAURANT SUPERV...| United States|            Stand-In|Mid senior|
|https://www.linke...|Independent Real ...| United States|   Real-Estate Clerk|Mid senior|
|https://www.linke...|Group/Unit Superv...| United States|Supervisor Travel...|Mid senior|
|https://www.linke...|  Counselor-07532802| United States| Residence Counselor|Mid senior|
|https://www.linke...|Registered Nurse ...| United States|  Nurse Practitioner|Mid senior|
|https://www.linke...|Part Time- HR Gen...| United States|Human Resource Ad...|Mid senior|

In [24]:
df_all.groupBy('job_title').count().orderBy('count', ascending=False).show()

+--------------------+-----+
|           job_title|count|
+--------------------+-----+
|LEAD SALES ASSOCI...| 7305|
|       Shift Manager| 5727|
|First Year Tax Pr...| 5350|
|LEAD SALES ASSOCI...| 4888|
|   Assistant Manager| 4606|
|Customer Service ...| 4599|
|CUSTOMER SERVICE ...| 4195|
|       Store Manager| 3929|
|Hourly Supervisor...| 2941|
|Travel Allied Hea...| 2920|
|    Registered Nurse| 2783|
|                Host| 2779|
|Hourly Supervisor...| 2227|
|  Restaurant Manager| 2135|
|OPERATIONS ASSIST...| 2062|
|Assistant General...| 1952|
|        Veterinarian| 1947|
|   Senior Accountant| 1914|
|   Account Executive| 1824|
|Department Superv...| 1685|
+--------------------+-----+
only showing top 20 rows



In [25]:
import re

pattern_danalyst = r'\bdata\sanal\w*'
pattern_banalyst = r'\bbusiness\sanal\w*'
#pattern_analyst = r'\banal\w*'
pattern_scientist = r'\bdata\sscientist\w*'
pattern_engineer = r'\bdata\sengin\w*'

In [28]:
from pyspark.sql.functions import col, lower, regexp_extract

df_all_sampled = df_all.filter(
    (lower(col('job_title')).rlike(pattern_danalyst)) |
    (lower(col('job_title')).rlike(pattern_banalyst)) |
#    (lower(col('job_title')).rlike(pattern_analyst)) |
    (lower(col('job_title')).rlike(pattern_scientist)) |
    (lower(col('job_title')).rlike(pattern_engineer))
)
print((df_all_sampled.count(), len(df_all_sampled.columns)))

(7038, 5)


In [29]:
df_all_sampled.show(10)

+--------------------+--------------------+--------------+--------------------+----------+
|            job_link|           job_title|search_country|     search_position| job_level|
+--------------------+--------------------+--------------+--------------------+----------+
|https://www.linke...|Secret cleared Bu...| United States|Quality Assurance...|Mid senior|
|https://www.linke...|Business Analyst ...| United States|Consultant Education|Mid senior|
|https://www.linke...|Payment (Swift) (...| United States|Instructor Busine...|Mid senior|
|https://www.linke...|Principal Busines...| United States|  Management Analyst|Mid senior|
|https://www.linke...|  Lead Data Engineer| United States|Maintenance Data ...|Mid senior|
|https://www.linke...|Senior Data Engineer| United States|Data Base Adminis...|Mid senior|
|https://www.linke...|    Business Analyst| United States| Instructor Modeling|Mid senior|
|https://www.linke...|SalesForce Busine...| United States|User Support Analyst|Mid senior|

In [32]:
new_names = ['link', 'job_title', 'country', 'serach', 'level']

df_all_sampled = df_all_sampled.toDF(*new_names)
df_all_sampled.limit(5).toPandas()

,link,job_title,country,serach,level
0,https://www.linkedin.com/jobs/view/secret-clea...,Secret cleared Business Analyst,United States,Quality Assurance Analyst,Mid senior
1,https://www.linkedin.com/jobs/view/business-an...,Business Analyst (11076-1),United States,Consultant Education,Mid senior
2,https://www.linkedin.com/jobs/view/payment-swi...,Payment (Swift) (Business Analyst),United States,Instructor Business Education,Mid senior
3,https://www.linkedin.com/jobs/view/principal-b...,Principal Business Analyst,United States,Management Analyst,Mid senior
4,https://www.linkedin.com/jobs/view/lead-data-e...,Lead Data Engineer,United States,Maintenance Data Analyst,Mid senior


Datasets merging a.k.a. sampling

In [34]:
data = df_all_sampled.join(df_skills,['link'],how='inner')
data.count()

6795

In [35]:
data.show()

+--------------------+--------------------+-------------+--------------------+----------+--------------------+
|                link|           job_title|      country|              serach|     level|               skill|
+--------------------+--------------------+-------------+--------------------+----------+--------------------+
|https://www.linke...|AFLCMC - Business...|United States|Consultant Education|Mid senior|Business Analyst,...|
|https://www.linke...|Business Analyst ...|United States|Instructor Busine...|Mid senior|Data analysis, SQ...|
|https://www.linke...|Business Analyst ...|United States|  Management Analyst|Mid senior|Business Analysis...|
|https://www.linke...|Business analyst ...|United States|            Designer|Mid senior|Alteryx Analytics...|
|https://www.linke...|    Business Analyst|United States|  Management Analyst|Mid senior|Data Analysis, Pr...|
|https://www.linke...|    Business Analyst|United States|Consultant Education|Mid senior|Data Repository, ...|
|

In [36]:
data = data[['link', 'skill']]
data.show(5)

+--------------------+--------------------+
|                link|               skill|
+--------------------+--------------------+
|https://www.linke...|Business Analyst,...|
|https://www.linke...|Data analysis, SQ...|
|https://www.linke...|Business Analysis...|
|https://www.linke...|Alteryx Analytics...|
|https://www.linke...|Data Analysis, Pr...|
+--------------------+--------------------+
only showing top 5 rows



# BMA

In [40]:
col = data.select('skill')
rdd = col.rdd
rdd = rdd.map(lambda x: x['skill'])
rdd.first()

"Business Analyst, DoD financial management processes, Government financial and accounting systems, Acquisition and budgeting processes, Relational databases, Financial and technical consulting, Government finance expertise, Financial data structuring, Reconciliation between systems, Financial forecasts and accounting data review, Government financial office support, Cost and Requirement System (CCaR™) experience, Bachelor's degree in business management engineering computer science mathematics economics, 7+ years of government acquisition finance financial management program management or related experience, 10+ years of equivalent technical training or work/military experience, Extended periods of sitting typing hearing seeing and speaking, Prolonged computer work and typing, Accurate communication, 15 lbs lifting/carrying, Office mobility, Federal background investigation, Travel, Varying work schedule, Additional hours and weekend work, $80000  $120000 annual salary, Employerpaid h

In [41]:
rdd.getNumPartitions()

200

In [42]:
baskets = rdd.map(lambda line: line.split(', '))
baskets.first()

['Business Analyst',
 'DoD financial management processes',
 'Government financial and accounting systems',
 'Acquisition and budgeting processes',
 'Relational databases',
 'Financial and technical consulting',
 'Government finance expertise',
 'Financial data structuring',
 'Reconciliation between systems',
 'Financial forecasts and accounting data review',
 'Government financial office support',
 'Cost and Requirement System (CCaR™) experience',
 "Bachelor's degree in business management engineering computer science mathematics economics",
 '7+ years of government acquisition finance financial management program management or related experience',
 '10+ years of equivalent technical training or work/military experience',
 'Extended periods of sitting typing hearing seeing and speaking',
 'Prolonged computer work and typing',
 'Accurate communication',
 '15 lbs lifting/carrying',
 'Office mobility',
 'Federal background investigation',
 'Travel',
 'Varying work schedule',
 'Additional

In [46]:
lens = baskets.map(lambda x: len(x))
print(lens.max())
print(lens.mean())
print(lens.mean() * data.count())

224
26.54598969830757


NameError: name 'mean_len' is not defined

In [48]:
print(lens.mean() * data.count())

180379.99999999994


In [49]:
supp = round((baskets.count())/50)
supp

136

## A-priory

## PCY

# Check with all-done algo

https://www.youtube.com/watch?v=guVvtZ7ZClw

https://www.kaggle.com/code/maricinnamon/market-basket-analysis-for-beginners

In [52]:
import numpy as np
import pandas as pd
import squarify
import matplotlib.pyplot as plt

# for market basket analysis
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

In [53]:
df = data.toPandas()
df.head(5)

,link,skill
0,https://www.linkedin.com/jobs/view/aflcmc-busi...,"Business Analyst, DoD financial management pro..."
1,https://www.linkedin.com/jobs/view/business-an...,"Data analysis, SQL, Database management, Macro..."
2,https://www.linkedin.com/jobs/view/business-an...,"Business Analysis, Business Process Review, Re..."
3,https://www.linkedin.com/jobs/view/business-an...,"Alteryx Analytics, SQL, Data Validation, Alter..."
4,https://www.linkedin.com/jobs/view/business-an...,"Data Analysis, Process Improvement, Technology..."


In [60]:
df0 = df.skill.str.split(', ', expand=True).transpose()
df0.head()

,0,1,2,3,4,5,6,7,8,9,...,6785,6786,6787,6788,6789,6790,6791,6792,6793,6794
0,Business Analyst,Data analysis,Business Analysis,Alteryx Analytics,Data Analysis,Data Repository,Agile methodology,Business Analysis,Data analysis,Data Science,...,Data analysis,JDE,Business Analysis,Pharma Commercial experience,Salesforce,Data Analysis,Data Automation,AWS,Data Analysis,Data Analysis
1,DoD financial management processes,SQL,Business Process Review,SQL,Process Improvement,Data Catalog,Business analysis,Data Analytics,Data modeling,Analytics,...,Data management,JDE E1 Order to Cash,SAP,IQVIA,Sales operations,Data Visualization,Data Engineering,SQL,Problem Formulation,Reporting
2,Government financial and accounting systems,Database management,Requirements Gathering and Documentation,Data Validation,Technology Evaluation and Selection,Data Classification,Business requirements,Statistical Analysis,Statistical analysis,Visualization,...,Statistical analysis,JDE E1 Procure to Pay,Oracle,AXTRIA,Sales enablement tools,Prototyping,ETL Development,Python,Statistics,Data Visualization
3,Acquisition and budgeting processes,Macro programming,Agile Scrum Methodology,Alteryx Designer,Stakeholder Collaboration,Requirement Elicitation,Communication skills,Forecasting,ETL,Machine Learning,...,Data visualization,JDE E1 financials,Nautical Systems,MuSigma,Business analysis,SQL,Data Wrangling,Linux,Machine Learning,SQL
4,Relational databases,Financial services knowledge,User Story Creation,Alteryx Server,Business Process Reengineering,Business Requirements,Data modeling,Report Writing,Data warehousing,Artificial Intelligence,...,SAS,AR,Software Development,WNS,Excel,Python,Robotic Process Automation,Relational DB / NoSQL,Python,Workday


In [ ]:
df_res = pd.DataFrame()
for i in range(len(df0.columns)):
    df_res = df_res._append(df0[i].value_counts())
df_res.head(3)

In [ ]:
df_sum = df_res.sum()
df_sum = df_sum.sort_values(ascending=False)

df_sum

In [ ]:
plt.figure(figsize=(14,8))
plt.title("FREQUENCY PLOT")
cnt = 45 # plot only first 'cnt' values
color = plt.cm.spring(np.linspace(0, 1, cnt))
df_sum.head(cnt).plot.bar(color = color)
plt.xticks(rotation = 'vertical')
plt.grid(False)
plt.axis('on')
plt.show()

In [ ]:
arr = []
for i in range(df0.shape[0]):
    arr.append([str(df0.values[i,j]) for j in range(df0.shape[1])])

arr = np.array(arr)
print(arr.shape)

In [ ]:
te = TransactionEncoder()
data = te.fit_transform(arr)
data = pd.DataFrame(data, columns = te.columns_)
print(data.shape)

In [ ]:
data = data.drop(columns=['nan'])
data.head(3)

In [ ]:
freq_rules = apriori(data, min_support = 0.01, use_colnames = True)
freq_rules

In [ ]:
freq_rules['length'] = freq_rules['itemsets'].apply(lambda x: len(x)) # adding 'length' column
freq_rules

In [ ]:
mask = freq_rules['length'] > 1 # creating mask for filtering with certain condition
filtered_freq_rules = freq_rules.loc[mask] # applying mask
filtered_freq_rules # printing the filtering result